In [1]:
import logging
import os
import sys
from dataclasses import dataclass, field
from typing import Optional

import datasets
from datasets import load_dataset, load_metric

import transformers
# from trainer_qa import QuestionAnsweringTrainer
from transformers import (
    AutoConfig,
    AutoModelForQuestionAnswering,
    AutoTokenizer,
    DataCollatorWithPadding,
    EvalPrediction,
    HfArgumentParser,
    PreTrainedTokenizerFast,
    TrainingArguments,
    default_data_collator,
    set_seed,
)
from transformers.trainer_utils import get_last_checkpoint
from transformers.utils import check_min_version
from transformers.utils.versions import require_version
# from utils_qa import postprocess_qa_predictions

/Users/chaeeunlee/anaconda3/envs/mediqa/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
import pandas as pd
import matplotlib.pyplot as plt
from wordcloud import WordCloud
import os
pd.set_option('display.max_colwidth', None)
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

# Specify the path to your CSV file
# csv_file_path = "/Users/chaeeunlee/Documents/VSC_workspaces/mediqa/data/Feb_1_2024_MS_Train_Val_Datasets/MEDIQA-CORR-2024-MS-TrainingData.csv"
# csv_file_path = "/Users/chaeeunlee/Documents/VSC_workspaces/mediqa/outputs/gpt3_api/2024-02-17/09-30-38/annotate_output_train.csv"
# csv_file_path = "/Users/chaeeunlee/Documents/VSC_workspaces/mediqa/outputs/gpt3_api/2024-02-17/13-00-05/detect_output_train.csv"
# csv_file_path = "/Users/chaeeunlee/Documents/VSC_workspaces/mediqa/outputs/gpt3_api/2024-02-17/14-42-38/detect_output_train.csv"
# csv_file_path = "/Users/chaeeunlee/Documents/VSC_workspaces/mediqa/outputs/gpt3_api/2024-02-17/14-42-38/detect_output_train-100-sampled.csv"

csv_file_path = "/Users/chaeeunlee/Documents/VSC_workspaces/mediqa/data/Feb_1_2024_MS_Train_Val_Datasets/MEDIQA-CORR-2024-MS-ValidationSet-1-Full.csv"
# csv_file_path ="/Users/chaeeunlee/Documents/VSC_workspaces/mediqa/outputs/gpt3_api/2024-02-17/09-30-38/annotate_output_train.csv"
# csv_file_path = "/Users/chaeeunlee/Documents/VSC_workspaces/mediqa/data/Feb_5_2024_UW_Validation_Set_Updated/MEDIQA-CORR-2024-UW-ValidationSet-1-Full_Feb.csv"
# a_correct = "/Users/chaeeunlee/Documents/VSC_workspaces/mediqa/outputs/gpt3_api/2024-02-22/12-58-48/detect_output_train_a_correct.csv"

df = pd.read_csv(csv_file_path)

# display(df.head(5))
print(df.columns)
df.shape

# ['Unnamed: 0', 'Text ID', 'Text', 'Sentences', 'Error Flag',
# 'Error Sentence ID', 'Error Sentence', 'Corrected Sentence',
# 'Corrected Text', 'prompt', 'sys_prompt',
# 'factual history of patient illness', 'medical exam results',
# 'interpretation of exam results', 'diagnoses', 'treatments',
######
# 'Prompt for interpretation of exam results 0',
# 'Sys Prompt for interpretation of exam results 0',
# 'Step-by-step Reasoning for interpretation of exam results 0',
# 'Final Answer for interpretation of exam results 0',

Index(['Unnamed: 0', 'Text ID', 'Text', 'Sentences', 'Error Flag',
       'Error Sentence ID', 'Error Sentence', 'Corrected Sentence',
       'Corrected Text'],
      dtype='object')


(574, 9)

In [3]:
csv_file_path = "/Users/chaeeunlee/Documents/VSC_workspaces/mediqa/data/Feb_5_2024_UW_Validation_Set_Updated/MEDIQA-CORR-2024-UW-ValidationSet-1-Full_Feb.csv"

df = pd.read_csv(csv_file_path)

# display(df.head(5))
print(df.columns)
df.shape

Index(['Unnamed: 0', 'Text ID', 'Text', 'Sentences', 'Error Flag',
       'Error Sentence ID', 'Error Sentence', 'Corrected Sentence',
       'Corrected Text'],
      dtype='object')


(160, 9)

In [3]:
df['question'] = "Which part in the given clinical note is clinically incorrect?"

# df.head(5)

In [4]:
# df['Text'] = df['Text'].apply(lambda x: x[0] if x else None)

In [5]:
import pandas as pd

def find_error_span(sentence1, sentence2):

    # has an error here. 
    differing_parts_sentence1 = []
    differing_parts_sentence2 = []

    if pd.isnull(sentence1):
        sentence1_words = []
    else:
        sentence1_words = sentence1.split()

    if pd.isnull(sentence2):
        sentence2_words = []
    else:
        sentence2_words = sentence2.split()

    for word in sentence1_words:
        if word not in sentence2_words:
            differing_parts_sentence1.append(word)

    for word in sentence2_words:
        if word not in sentence1_words:
            differing_parts_sentence2.append(word)

    return " ".join(differing_parts_sentence1), " ".join(differing_parts_sentence2)


def find_error_span(sentence1, sentence2):
    # Check if sentences are null and split them into words
    sentence1_words = [] if pd.isnull(sentence1) else sentence1.split()
    sentence2_words = [] if pd.isnull(sentence2) else sentence2.split()

    # Initialize indices lists
    diff_word_idx_sent1 = [idx for idx, word in enumerate(sentence1_words) if word not in sentence2_words]
    diff_word_idx_sent2 = [idx for idx, word in enumerate(sentence2_words) if word not in sentence1_words]

    # Identify contiguous differing span in sentence1
    if diff_word_idx_sent1:
        differing_parts_sentence1 = sentence1_words[min(diff_word_idx_sent1):max(diff_word_idx_sent1)+1]
    else:
        differing_parts_sentence1 = []

    # Identify contiguous differing span in sentence2
    if diff_word_idx_sent2:
        differing_parts_sentence2 = sentence2_words[min(diff_word_idx_sent2):max(diff_word_idx_sent2)+1]
    else:
        differing_parts_sentence2 = []

    return " ".join(differing_parts_sentence1), " ".join(differing_parts_sentence2)

# # Example usage
# sentence1 = "This is an exciting example sentence."
# sentence2 = "This is a exciting sample sentence."
# print(find_error_span(sentence1, sentence2))


df['Error Span'], df['Corrected Span'] = None, None

# df.loc[df['Corrected Sentence'].isna(), 'Correction'] = "NA"

res = df.apply(lambda row: find_error_span(row['Error Sentence'], row['Corrected Sentence']), axis=1)
# df['Correction'] = df.apply(lambda row: find_error_span(row['Error Sentence'], row['Corrected Sentence']) if pd.notnull(row['Corrected Sentence']) else None, axis=1)

df['Error Span'] = res.apply(lambda x: x[0]) # the fuck is this syntax? -> ah so res is two-column df and this one's just taking the first col and adding that to df. 
df['Corrected Span'] = res.apply(lambda x: x[1])

In [6]:
df.loc[570, "Error Span"] # ''
len(df.loc[570, "Error Span"]) # 0

0

In [7]:
def find_all_subsequences(text, sub_sequence, verbose=True):

    if len(sub_sequence)==0:
        return []
        

    start_indices = []
    start_index = text.find(sub_sequence)
    
    while start_index != -1:
        start_indices.append(start_index)
        # Move the search window to look for the next occurrence
        start_index = text.find(sub_sequence, start_index + 1)
        
        # Check if the subsequence is found more than twice -> more than once!
        if len(start_indices) > 1:
            error_msg = f"The subsequence '{sub_sequence}' is found more than once in the text."
            if verbose:
                error_msg = error_msg + f"\nText:\n{text}" + "\n------------------------------------------"
            # raise ValueError(error_msg)
            print(error_msg)

    try:
        res = [start_indices[0]]
    except:
        print(f"hmm {sub_sequence}, {text}")
    
    return res

# Example usage
text = "This is a sample text containing a certain subsequence and another certain subsequence."
sub_sequence = "certain subsequence"

try:
    start_indices = find_all_subsequences(text, sub_sequence)
    print(f"The start indices of the subsequence are: {start_indices}")
except ValueError as e:
    print(e)


The subsequence 'certain subsequence' is found more than once in the text.
Text:
This is a sample text containing a certain subsequence and another certain subsequence.
------------------------------------------
The start indices of the subsequence are: [35]


In [8]:
# we need new columns: 

# raw_datasets = load_dataset(data_args.dataset_name, data_args.dataset_config_name, cache_dir=model_args.cache_dir)

# import pandas as pd
from datasets import Dataset

hf_dataset = Dataset.from_pandas(df)



In [9]:
# answers = examples[answer_column_name][sample_index]
# if len(answers["answer_start"]) == 0:

# hf_dataset['Error Span'] = 

# df['answers'] = df.apply(lambda row: find_all_subsequences(text=row['Text'], sub_sequence=row['Error Span']), axis=1)

def transform_error_span(example):
    original_error_span = example['Error Span']
    return {
        "Answer": {
            "text": original_error_span,
            "answer_start": find_all_subsequences(text=example['Text'], sub_sequence=original_error_span)
        }
    }

# Apply the transformation to each row in the dataset
hf_dataset = hf_dataset.map(transform_error_span)

Map:   0%|          | 0/574 [00:00<?, ? examples/s]

Map: 100%|██████████| 574/574 [00:00<00:00, 14107.34 examples/s]

The subsequence 'a disc herniation.' is found more than once in the text.
Text:
A 14-year-old boy is brought to the physician for the evaluation of back pain for the past six months. The pain is worse with exercise and when reclining. He attends high school and is on the swim team. He also states that he lifts weights on a regular basis. He has not had any trauma to the back or any previous problems with his joints. He has no history of serious illness. His father has a disc herniation. Palpation of the spinous processes at the lumbosacral area shows that two adjacent vertebrae are displaced and are at different levels. Muscle strength is normal. Sensation to pinprick and light touch is intact throughout. When the patient is asked to walk, a waddling gait is noted. Passive raising of either the right or left leg causes pain radiating down the ipsilateral leg. Patient was diagnosed with a disc herniation. 
------------------------------------------
The subsequence 'The' is found more th

In [10]:
print(hf_dataset)
print(hf_dataset['Answer']) # {'answer_start': [], 'text': ''}

Dataset({
    features: ['Unnamed: 0', 'Text ID', 'Text', 'Sentences', 'Error Flag', 'Error Sentence ID', 'Error Sentence', 'Corrected Sentence', 'Corrected Text', 'question', 'Error Span', 'Corrected Span', 'Answer'],
    num_rows: 574
})
[{'answer_start': [457], 'text': 'Neisseria gonorrhoeae.'}, {'answer_start': [], 'text': ''}, {'answer_start': [997], 'text': 'Streptococcus'}, {'answer_start': [436], 'text': 'a respiratory syncytial virus infection.'}, {'answer_start': [77], 'text': 'Bartonella henselae'}, {'answer_start': [], 'text': ''}, {'answer_start': [531], 'text': 'vibrio cholerae'}, {'answer_start': [], 'text': ''}, {'answer_start': [691], 'text': 'paramyxovirus.'}, {'answer_start': [], 'text': ''}, {'answer_start': [593], 'text': 'aureus.'}, {'answer_start': [], 'text': ''}, {'answer_start': [608], 'text': 'Sporothrix schenckii'}, {'answer_start': [], 'text': ''}, {'answer_start': [812], 'text': 'Mycoplasma genitalium.'}, {'answer_start': [], 'text': ''}, {'answer_start': 

In [11]:
print(df.columns)

display(df.head(1))

Index(['Unnamed: 0', 'Text ID', 'Text', 'Sentences', 'Error Flag',
       'Error Sentence ID', 'Error Sentence', 'Corrected Sentence',
       'Corrected Text', 'question', 'Error Span', 'Corrected Span'],
      dtype='object')


,Unnamed: 0,Text ID,Text,Sentences,Error Flag,Error Sentence ID,Error Sentence,Corrected Sentence,Corrected Text,question,Error Span,Corrected Span
0,0,ms-val-0,"A 24-year-old woman comes to the emergency department because of a 4-hour history of headaches, nausea, and vomiting. During this time, she has also had recurrent dizziness and palpitations. The symptoms started while she was at a friend's birthday party, where she had one beer. One week ago, the patient was diagnosed with a genitourinary infection and started on antimicrobial therapy. She has no history of major medical illness. Culture tests indicate Neisseria gonorrhoeae. Her pulse is 106/min and blood pressure is 102/73 mm Hg. Physical examination shows facial flushing and profuse sweating.","0 A 24-year-old woman comes to the emergency department because of a 4-hour history of headaches, nausea, and vomiting.\n1 During this time, she has also had recurrent dizziness and palpitations.\n2 The symptoms started while she was at a friend's birthday party, where she had one beer.\n3 One week ago, the patient was diagnosed with a genitourinary infection and started on antimicrobial therapy.\n4 She has no history of major medical illness.\n5 Culture tests indicate Neisseria gonorrhoeae.\n6 Her pulse is\n7 106/min and blood pressure is 102/73 mm\n8 Hg.\n9 Physical examination shows facial flushing and profuse sweating.",1,5,Culture tests indicate Neisseria gonorrhoeae.,Culture tests indicate Trichomonas vaginalis.,"A 24-year-old woman comes to the emergency department because of a 4-hour history of headaches, nausea, and vomiting. During this time, she has also had recurrent dizziness and palpitations. The symptoms started while she was at a friend's birthday party, where she had one beer. One week ago, the patient was diagnosed with a genitourinary infection and started on antimicrobial therapy. She has no history of major medical illness. Culture tests indicate Trichomonas vaginalis. Her pulse is 106/min and blood pressure is 102/73 mm Hg. Physical examination shows facial flushing and profuse sweating.",Which part in the given clinical note is clinically incorrect?,Neisseria gonorrhoeae.,Trichomonas vaginalis.


In [12]:
# dir(hf_dataset)

In [13]:
# max_seq_length,
# data_args.doc_stride,
# data_args.do_not_use_token_type_ids:

# answer_column_name -> we have to create span column

# what exactly are we giving as examples? i think it's a df? or dictionary from json. 
# train_dataset = train_dataset.map(
#                 prepare_train_features, -> ah so it's a HF dataset. that's fine. 

In [14]:
@dataclass
class ModelArguments:
    """
    Arguments pertaining to which model/config/tokenizer we are going to fine-tune from.
    """

    model_name_or_path: str = field(
        default=None,
        metadata={"help": "Path to pretrained model or model identifier from huggingface.co/models"}
    )
    config_name: Optional[str] = field(
        default=None, metadata={"help": "Pretrained config name or path if not the same as model_name"}
    )
    tokenizer_name: Optional[str] = field(
        default=None, metadata={"help": "Pretrained tokenizer name or path if not the same as model_name"}
    )
    cache_dir: Optional[str] = field(
        default=None,
        metadata={"help": "Path to directory to store the pretrained models downloaded from huggingface.co"},
    )
    model_revision: str = field(
        default="main",
        metadata={"help": "The specific model version to use (can be a branch name, tag name or commit id)."},
    )
    use_auth_token: bool = field(
        default=False,
        metadata={
            "help": "Will use the token generated when running `transformers-cli login` (necessary to use this script "
            "with private models)."
        },
    )

# parser = HfArgumentParser((ModelArguments,))# DataTrainingArguments, TrainingArguments))
# if len(sys.argv) == 2 and sys.argv[1].endswith(".json"):
#     model_args, data_args, training_args = parser.parse_json_file(json_file=os.path.abspath(sys.argv[1]))
# else:
#     model_args, data_args, training_args = parser.parse_args_into_dataclasses()

In [15]:
@dataclass
class DataTrainingArguments:
    """
    Arguments pertaining to what data we are going to input our model for training and eval.
    """

    dataset_name: Optional[str] = field(
        default=None, metadata={"help": "The name of the dataset to use (via the datasets library)."}
    )
    dataset_config_name: Optional[str] = field(
        default=None, metadata={"help": "The configuration name of the dataset to use (via the datasets library)."}
    )
    train_file: Optional[str] = field(default=None, metadata={"help": "The input training data file (a text file)."})
    validation_file: Optional[str] = field(
        default=None,
        metadata={"help": "An optional input evaluation data file to evaluate the perplexity on (a text file)."},
    )
    test_file: Optional[str] = field(
        default=None,
        metadata={"help": "An optional input test data file to evaluate the perplexity on (a text file)."},
    )
    overwrite_cache: bool = field(
        default=False, metadata={"help": "Overwrite the cached training and evaluation sets"}
    )
    preprocessing_num_workers: Optional[int] = field(
        default=None,
        metadata={"help": "The number of processes to use for the preprocessing."},
    )
    do_not_use_token_type_ids:  bool = field(
        default=False, metadata={"help": "If true, do not use token_type_ids."}
    )
    max_seq_length: int = field(
        default=384,
        metadata={
            "help": "The maximum total input sequence length after tokenization. Sequences longer "
            "than this will be truncated, sequences shorter will be padded."
        },
    )
    pad_to_max_length: bool = field(
        default=True,
        metadata={
            "help": "Whether to pad all samples to `max_seq_length`. "
            "If False, will pad the samples dynamically when batching to the maximum length in the batch (which can "
            "be faster on GPU but will be slower on TPU)."
        },
    )
    max_train_samples: Optional[int] = field(
        default=None,
        metadata={
            "help": "For debugging purposes or quicker training, truncate the number of training examples to this "
            "value if set."
        },
    )
    max_eval_samples: Optional[int] = field(
        default=None,
        metadata={
            "help": "For debugging purposes or quicker training, truncate the number of evaluation examples to this "
            "value if set."
        },
    )
    max_predict_samples: Optional[int] = field(
        default=None,
        metadata={
            "help": "For debugging purposes or quicker training, truncate the number of prediction examples to this "
            "value if set."
        },
    )
    version_2_with_negative: bool = field(
        default=False, metadata={"help": "If true, some of the examples do not have an answer."}
    )
    null_score_diff_threshold: float = field(
        default=0.0,
        metadata={
            "help": "The threshold used to select the null answer: if the best answer has a score that is less than "
            "the score of the null answer minus this threshold, the null answer is selected for this example. "
            "Only useful when `version_2_with_negative=True`."
        },
    )
    doc_stride: int = field(
        default=128,
        metadata={"help": "When splitting up a long document into chunks, how much stride to take between chunks."},
    )
    n_best_size: int = field(
        default=50,
        metadata={"help": "The total number of n-best predictions to generate when looking for an answer."},
    )
    max_answer_length: int = field(
        default=30,
        metadata={
            "help": "The maximum length of an answer that can be generated. This is needed because the start "
            "and end predictions are not conditioned on one another."
        },
    )

In [16]:
from dataclasses import dataclass, field
from typing import Optional

# Assuming the ModelArguments class definition is as you've provided

# Now, to instantiate this class with some arguments:
model_args = ModelArguments(
    model_name_or_path="michiyasunaga/BioLinkBERT-large",
    config_name="michiyasunaga/BioLinkBERT-large",
    tokenizer_name="michiyasunaga/BioLinkBERT-large",
    # cache_dir='/path/to/your/cache/dir',
    # model_revision='main',
    # use_auth_token=True
)

data_args = DataTrainingArguments(
    max_seq_length = 512, # 384
)

In [17]:
# model_args.tokenizer_name = michiyasunaga/BioLinkBERT-large

tokenizer = AutoTokenizer.from_pretrained(
    model_args.tokenizer_name if model_args.tokenizer_name else model_args.model_name_or_path,
    # cache_dir=model_args.cache_dir,
    use_fast=True,
    # revision=model_args.model_revision,
    # use_auth_token=True if model_args.use_auth_token else None,
)

In [18]:
pad_on_right = tokenizer.padding_side == "right"

max_seq_length = min(data_args.max_seq_length, tokenizer.model_max_length)

In [19]:
# hmm yea... we don't have distinction between question and context... should i just come up with a random question?

question_column_name = "question" 
context_column_name = "Text" # the entire clinical text. 

answer_column_name = 'Answer' # print(hf_dataset['Answer']) # {'answer_start': [], 'text': ''}

In [20]:
# prepare_validation_features -> determines what kind of preprocessing is needed. 

# Training preprocessing
def prepare_train_features(examples):
    # Tokenize our examples with truncation and maybe padding, but keep the overflows using a stride. This results
    # in one example possible giving several features when a context is long, each of those features having a
    # context that overlaps a bit the context of the previous feature.
    tokenized_examples = tokenizer(
        examples[question_column_name if pad_on_right else context_column_name],
        examples[context_column_name if pad_on_right else question_column_name], # type(examples['Text']): list
        truncation="only_second" if pad_on_right else "only_first",
        max_length=max_seq_length,
        stride=data_args.doc_stride,
        return_overflowing_tokens=True,
        return_offsets_mapping=True,
        padding="max_length" if data_args.pad_to_max_length else False,
    )
    if data_args.do_not_use_token_type_ids:
        if "token_type_ids" in tokenized_examples:
            print ("drop token_type_ids!")
            tokenized_examples.pop("token_type_ids")
            assert "token_type_ids" not in tokenized_examples

    # Since one example might give us several features if it has a long context, we need a map from a feature to
    # its corresponding example. This key gives us just that.
    sample_mapping = tokenized_examples.pop("overflow_to_sample_mapping") # len(sample_mapping): 50 (bs)
    # The offset mappings will give us a map from token to character position in the original context. This will
    # help us compute the start_positions and end_positions.
    offset_mapping = tokenized_examples.pop("offset_mapping")

    # Let's label those examples!
    tokenized_examples["start_positions"] = []
    tokenized_examples["end_positions"] = []

    # len(tokenized_examples["input_ids"])
    # 50
    # len(tokenized_examples["input_ids"][0])
    # 384

    print(f"len(tokenized_examples['input_ids']) = {len(tokenized_examples['input_ids'])}")

    for i, offsets in enumerate(offset_mapping): # len(offset_mapping): 50 // len(offset_mapping[0]) : 384
        # We will label impossible answers with the index of the CLS token.
        input_ids = tokenized_examples["input_ids"][i] # len(tokenized_examples["input_ids"][0]): 384
        cls_index = input_ids.index(tokenizer.cls_token_id)

        # Grab the sequence corresponding to that example (to know what is the context and what is the question).
        sequence_ids = tokenized_examples.sequence_ids(i)

        # One example can give several spans, this is the index of the example containing this span of text.
        sample_index = sample_mapping[i]
        answers = examples[answer_column_name][sample_index]
        # If no answers are given, set the cls_index as answer.
        # import pdb; pdb.set_trace()
        if len(answers["answer_start"]) == 0:
            tokenized_examples["start_positions"].append(cls_index)
            tokenized_examples["end_positions"].append(cls_index)
        else:
            # Start/end character index of the answer in the text.
            start_char = answers["answer_start"][0]
            end_char = start_char + len(answers["text"][0])

            # Start token index of the current span in the text.
            token_start_index = 0
            while sequence_ids[token_start_index] != (1 if pad_on_right else 0):
                token_start_index += 1

            # End token index of the current span in the text.
            token_end_index = len(input_ids) - 1
            while sequence_ids[token_end_index] != (1 if pad_on_right else 0):
                token_end_index -= 1

            # Detect if the answer is out of the span (in which case this feature is labeled with the CLS index).
            if not (offsets[token_start_index][0] <= start_char and offsets[token_end_index][1] >= end_char):
                tokenized_examples["start_positions"].append(cls_index)
                tokenized_examples["end_positions"].append(cls_index)
            else:
                # Otherwise move the token_start_index and token_end_index to the two ends of the answer.
                # Note: we could go after the last offset if the answer is the last word (edge case).
                while token_start_index < len(offsets) and offsets[token_start_index][0] <= start_char:
                    token_start_index += 1
                tokenized_examples["start_positions"].append(token_start_index - 1)
                while offsets[token_end_index][1] >= end_char:
                    token_end_index -= 1
                tokenized_examples["end_positions"].append(token_end_index + 1)

    return tokenized_examples



In [21]:
# what exactly are we giving as examples? i think it's a df? or dictionary from json. 
# train_dataset = train_dataset.map(
#                 prepare_train_features, -> ah so it's a HF dataset. that's fine. 

        # with training_args.main_process_first(desc="train dataset map pre-processing"):
        #     train_dataset = train_dataset.map(
        #         prepare_train_features,
        #         batched=True,
        #         num_proc=data_args.preprocessing_num_workers,
        #         remove_columns=column_names,
        #         load_from_cache_file=not data_args.overwrite_cache,
        #         desc="Running tokenizer on train dataset",
        #     )

mapped_hf_dataset = hf_dataset.map(prepare_train_features, batched=True, batch_size=50) # the 52 thing fixed when i changed from 384 to 512 the max seq length. 

Parameter 'function'=<function prepare_train_features at 0x17f528280> of the transform datasets.arrow_dataset.Dataset._map_single couldn't be hashed properly, a random hash was used instead. Make sure your transforms and parameters are serializable with pickle or dill for the dataset fingerprinting and caching to work. If you reuse this transform, the caching mechanism will consider it to be different from the previous calls and recompute everything. This warning is only showed once. Subsequent hashing failures won't be showed.
Map: 100%|██████████| 574/574 [00:00<00:00, 3993.93 examples/s]

len(tokenized_examples['input_ids']) = 50
len(tokenized_examples['input_ids']) = 50
len(tokenized_examples['input_ids']) = 50
len(tokenized_examples['input_ids']) = 50
len(tokenized_examples['input_ids']) = 50
len(tokenized_examples['input_ids']) = 50
len(tokenized_examples['input_ids']) = 50
len(tokenized_examples['input_ids']) = 50
len(tokenized_examples['input_ids']) = 50
len(tokenized_examples['input_ids']) = 50
len(tokenized_examples['input_ids']) = 50
len(tokenized_examples['input_ids']) = 24


In [22]:
from datasets import load_dataset

# Load the SQuAD v2.0 dataset
dataset = load_dataset("squad_v2")

In [23]:
# dataset.column_names
# {'train': ['id', 'title', 'context', 'question', 'answers'],
#  'validation': ['id', 'title', 'context', 'question', 'answers']}

# dataset['train']['answers'][4]

In [24]:
# row_id = 4
row_id +=1

answers = dataset['train']['answers'][row_id]
context = dataset['train']['context'][row_id]


start_index = context.find(answers['text'][0])

print(f"start_index = {start_index}")
print(f"'answer_start' = {answers['answer_start']}")

NameError: name 'row_id' is not defined

In [ ]:
# model_args, data_args, training_args = parser.parse_args_into_dataclasses()

ValueError: Some specified arguments are not used by the HfArgumentParser: ['--f=/Users/chaeeunlee/Library/Jupyter/runtime/kernel-v2-3929uGTbTULPstDI.json']

In [ ]:
# if len(sys.argv) == 2 and sys.argv[1].endswith(".json"):
#     model_args, data_args, training_args = parser.parse_json_file(json_file=os.path.abspath(sys.argv[1]))
# else:
#     model_args, data_args, training_args = parser.parse_args_into_dataclasses()

In [ ]:
res = prepare_train_features(hf_dataset)

KeyboardInterrupt: 

In [ ]:
if training_args.do_train:
    if "train" not in raw_datasets:
        raise ValueError("--do_train requires a train dataset")
    train_dataset = raw_datasets["train"]
    if data_args.max_train_samples is not None:
        # We will select sample from whole data if agument is specified
        train_dataset = train_dataset.select(range(data_args.max_train_samples))
    # Create train feature from dataset
    with training_args.main_process_first(desc="train dataset map pre-processing"):
        train_dataset = train_dataset.map(
            prepare_train_features,
            batched=True,
            num_proc=data_args.preprocessing_num_workers,
            remove_columns=column_names,
            load_from_cache_file=not data_args.overwrite_cache,
            desc="Running tokenizer on train dataset",
        )
    if data_args.max_train_samples is not None:
        # Number of samples might increase during Feature Creation, We select only specified max samples
        train_dataset = train_dataset.select(range(data_args.max_train_samples))